In [6]:
import brainlit
from brainlit.utils.ngl_pipeline import NeuroglancerSession
from brainlit.viz.swc import *
from brainlit.algorithms.generate_fragments import adaptive_thresh, tube_seg

In [7]:
data_dir = "benchmarking_datasets/"

In [19]:
from skimage import io

In [20]:
im = io.imread("/Users/mattf/Documents/2020_Courses/NDD/benchmarking_datasets/test_1-gfp.tif")

In [10]:
!pwd

/Users/mattf/Documents/2020_Courses/NDD/mouselit/matt


In [11]:
# Reference: https://github.com/NeuroDataDesign/mouselit/blob/master/bijan/mouse_test/final%20notebook.ipynb

import matplotlib.pyplot as plt
import SimpleITK as sitk
import numpy as np
import napari


def plot_2d(img, title=None, margin=0.05, dpi=80):
    nda = sitk.GetArrayFromImage(img)
    spacing = img.GetSpacing()

    if nda.ndim == 3:
        c = nda.shape[-1]

        if c not in (3, 4):
            nda = nda[nda.shape[0] // 2, :, :]

    elif nda.ndim == 4:
        c = nda.shape[-1]

        if c not in (3, 4):
            raise RuntimeError("Unable to show 3D-vector Image")

        nda = nda[nda.shape[0] // 2, :, :, :]

    xsize = nda.shape[1] * 2
    ysize = nda.shape[0] * 2

    figsize = (1 + margin) * xsize / dpi, (1 + margin) * ysize / dpi

    plt.figure(figsize=figsize, dpi=dpi, tight_layout=True)
    ax = plt.gca()

    extent = (0, xsize * spacing[0], ysize * spacing[1], 0)

    t = ax.imshow(nda, extent=extent, interpolation=None)

    if nda.ndim == 2:
        t.set_cmap("gray")

    if title:
        plt.title(title)

    plt.show()


def plot_3d(img, xslices=[], yslices=[], zslices=[], title=None, margin=0.05, dpi=80):
    img_xslices = [img[s, :, :] for s in xslices]
    img_yslices = [img[:, s, :] for s in yslices]
    img_zslices = [img[:, :, s] for s in zslices]

    maxlen = max(len(img_xslices), len(img_yslices), len(img_zslices))

    img_null = sitk.Image([0, 0], img.GetPixelID(), img.GetNumberOfComponentsPerPixel())

    img_slices = []
    d = 0

    if len(img_xslices):
        img_slices += img_xslices + [img_null] * (maxlen - len(img_xslices))
        d += 1

    if len(img_yslices):
        img_slices += img_yslices + [img_null] * (maxlen - len(img_yslices))
        d += 1

    if len(img_zslices):
        img_slices += img_zslices + [img_null] * (maxlen - len(img_zslices))
        d += 1

    if maxlen != 0:
        if img.GetNumberOfComponentsPerPixel() == 1:
            img = sitk.Tile(img_slices, [maxlen, d])
        else:
            img_comps = []
            for i in range(0, img.GetNumberOfComponentsPerPixel()):
                img_slices_c = [sitk.VectorIndexSelectionCast(s, i) for s in img_slices]
                img_comps.append(sitk.Tile(img_slices_c, [maxlen, d]))
            img = sitk.Compose(img_comps)

    plot_2d(img, title, margin, dpi)


def napari_viewer(img, labels=None, shapes=None, label_name="Segmentation"):
    viewer = napari.view_image(np.squeeze(np.array(img)))
    if labels is not None:
        viewer.add_labels(labels, name=label_name)
    if shapes is not None:
        viewer.add_shapes(data=shapes, shape_type='path', edge_color='blue', name='Skeleton')
    return viewer

In [12]:
import numpy as np

In [23]:
%gui qt
napari_viewer(im)

In [25]:
im.shape

(100, 330, 330)

In [26]:
swc_path = "/Users/mattf/Documents/2020_Courses/NDD/benchmarking_datasets/Manual-GT/8-01_test_1-5/8-01_test_1/tree_1.swc"

In [27]:
import pandas as pd

In [41]:
df = pd.read_csv(swc_path, skiprows = 3, header = None, sep = " ")

In [49]:
subneuron_df = df[:15]

In [50]:
subneuron_df

,0,1,2,3,4,5,6
0,1,5,-54.779639,-12.908684,14.684862,1.0,-1
1,2,0,-54.480880,-12.604268,14.684862,1.0,1
2,3,0,-54.182121,-12.604268,14.684862,1.0,2
3,4,0,-53.883362,-12.299852,14.684862,1.0,3
4,5,0,-52.090806,-12.299852,14.684862,1.0,4
5,6,0,-50.895769,-12.299852,14.684862,1.0,5
6,7,0,-50.597010,-11.995436,14.684862,1.0,6
7,8,0,-49.700732,-11.995436,14.684862,1.0,7
8,9,0,-49.401973,-11.691020,14.684862,1.0,8
9,10,0,-49.103214,-11.691020,14.684862,1.0,9


In [51]:
subneuron_df.columns = ["sample",  "structure",      "x",      "y",      "z",
                         "r",  "parent"]

In [52]:
G = df_to_graph(subneuron_df)
print('Number of nodes:', len(G.nodes))
print('Number of edges:', len(G.edges))
print('\n')
print('Sample 1 coordinates (x,y,z)')
G.nodes[1]

Number of nodes: 15
Number of edges: 14


Sample 1 coordinates (x,y,z)


{'x': -54, 'y': -12, 'z': 14}

In [53]:
paths = graph_to_paths(G)
len(paths)

1

In [54]:
%gui qt
napari_viewer(im, shapes=paths)

In [62]:
seed = []
for index, row in subneuron_df.iterrows():
    sample = (row['x'], row['y'], row['z'])
    _, s = adaptive_thresh.get_seed(sample)
    seed.append(s)
print(seed)

[(14, -12, -54), (14, -12, -54), (14, -12, -54), (14, -12, -53), (14, -12, -52), (14, -12, -50), (14, -11, -50), (14, -11, -49), (14, -11, -49), (14, -11, -49), (14, -11, -48), (13, -11, -47), (13, -11, -47), (12, -11, -47), (12, -11, -46)]


In [ ]:
labels = adaptive_thresh.confidence_connected_threshold(img, seed, num_iter=1, multiplier=0.3)

In [ ]:
## Most useful visualization cell - contains image, SWC, and automatic segmentation
%gui qt
napari_viewer(img, labels=labels, shapes=paths, label_name="Confidence-Connected Threshold")

In [ ]:
labels = tube_seg.tubes_seg(img, vox_in_img_list, 2)

In [ ]:
## Most useful visualization cell - contains image, SWC, and automatic segmentation
%gui qt
napari_viewer(img, labels=labels, shapes=paths, label_name="Tube Segmentation")